# Load Helper Functions

In [1]:
from util import get_service_area, load_Baltimore_blocks
import geopandas as gpd
import pickle
import datetime
import folium
from r5py import TransportNetwork, DetailedItinerariesComputer, TransportMode
import warnings
warnings.filterwarnings('ignore')

gdf_Baltimore, md_rac_df, md_wac_df = load_Baltimore_blocks()
stations = gpd.read_file(f'../../raw_data/Baltimore Red Line- Stations.csv')
segments = gpd.read_file(f'../../raw_data/Baltimore Red Line- Alignment.csv')
service_area = get_service_area(gdf_Baltimore, stations)
residents = service_area.merge(md_rac_df, left_on='GEOID20', right_on='h_geocode')
#pickle.dump(residents, open('../../processed_data/residents.pkl', 'wb'))
workplace = pickle.load(open('../../processed_data/workplace.pkl', 'rb'))
osm_fp = '../../raw_data/maryland-latest.osm.pbf'
GTFS = ['../../processed_data/updated_gtfs/2023-04-10.zip', '../../raw_data/commuterbus_gtfs.zip', '../../raw_data/subway_gtfs.zip',
                  '../../raw_data/train_gtfs.zip', '../../raw_data/rail_gtfs.zip']

transport_network_before_newline = TransportNetwork(
        osm_fp,
        GTFS
    )

GTFS = ['../../processed_data/updated_gtfs/2023-04-10.zip', '../../raw_data/commuterbus_gtfs.zip', '../../raw_data/subway_gtfs.zip',
                  '../../raw_data/train_gtfs.zip', '../../raw_data/rail_gtfs.zip', '../../processed_data/redline.zip']

transport_network_after_newline = TransportNetwork(
        osm_fp,
        GTFS
    )

In [2]:
def compute_itineraries(transport_network, origin, destination, center = [39.2900, -76.6480]):

    origins = residents[residents['GEOID20'] == origin]
    origins['id'] = origins['GEOID20']
    origins = origins[['geometry', 'id']]

    destinations = workplace[workplace['GEOID20'] == destination]
    destinations['id'] = destinations['GEOID20']
    destinations = destinations[['geometry', 'id']]
    
    origins.geometry = origins.geometry.centroid
    destinations.geometry = destinations.geometry.centroid

    detailed_itineraries_computer = DetailedItinerariesComputer(
            transport_network,
            origins=origins,
            destinations=destinations,
            departure=datetime.datetime(2023, 4, 10, 8, 0),
            transport_modes=[TransportMode.TRANSIT, TransportMode.WALK],
            departure_time_window = datetime.timedelta(minutes=10)
        )

    travel_details = detailed_itineraries_computer.compute_travel_details()

    travel_details["mode"] = travel_details.transport_mode.astype(str)
    travel_details["travel time (min)"] = travel_details.travel_time.apply(
        lambda t: round(t.total_seconds() / 60.0, 2)
    )

    travel_details["wait time (min)"] = travel_details.wait_time.apply(
        lambda t: round(t.total_seconds() / 60.0, 2)
    )


    options = travel_details.groupby("option")["travel time (min)"].sum() + travel_details.groupby("option")["wait time (min)"].sum()

    travel_details = travel_details[travel_details['option'] == options.idxmin()]

    city_map = folium.Map(location=center, zoom_start=13)

    color_dict = {
        'TransportMode.BUS': 'blue',
        'TransportMode.SUBWAY': 'gray',
        'TransportMode.TRAM': 'red',
        'TransportMode.WALK': 'green',
    }


    for _, station in origins.iterrows():
        folium.Marker(
            location=[station.geometry.y, station.geometry.x],  # Extract latitude and longitude from the geometry
            icon=folium.Icon(color='red'),  # Specify the custom icon,
        ).add_to(city_map)


    for _, station in destinations.iterrows():
        folium.Marker(
            location=[station.geometry.y, station.geometry.x],  # Extract latitude and longitude from the geometry
            icon=folium.Icon(color='green'),  # Specify the custom icon,
        ).add_to(city_map)

    for _, route in travel_details.iterrows():
        folium.PolyLine(
            locations=[(lat, lon) for lon, lat in route['geometry'].coords],
            color=color_dict[route['mode']],
            weight=3,
            opacity=1,
            dash_array='5, 5',
            tooltip=route['mode']
        ).add_to(city_map)

    colors = [color_dict[mode] for mode in travel_details['mode'].unique()]
    labels = [mode for mode in travel_details['mode'].unique()]
    legend_html = '<div style="background: white; width: 200px; height: 70px; border: 1px solid black;font-size: 16px;">'
    for i in range(len(colors)):
        legend_html += f"""
        <div><span style="background-color: {colors[i]}; 
                        width: 10px; 
                        height: 10px; 
                        display: inline-block; 
                        margin-left: 5px; margin-right: 5px;"></span >{labels[i]}</div>"""
    legend_html += '</div>'

    legend_icon = folium.DivIcon(html=legend_html)
    return options, travel_details, city_map

# Aisha
Aisha travels from block 245101607002002 to 245100402001036

In [3]:
options, travel_details, city_map = compute_itineraries(transport_network_before_newline, '245101607002002', '245100402001036')
print(options.min())
travel_details

45.58


,from_id,to_id,option,segment,transport_mode,departure_time,distance,travel_time,wait_time,route,geometry,mode,travel time (min),wait time (min)
16,245101607002002,245100402001036,4,0,TransportMode.WALK,2023-04-10 08:06:15,156.667,0 days 00:02:34,0 days 00:00:00,None,"LINESTRING (-76.66697 39.30002, -76.66587 39.3...",TransportMode.WALK,2.57,0.00
17,245101607002002,245100402001036,4,1,TransportMode.BUS,2023-04-10 08:12:41,NaN,0 days 00:07:53,0 days 00:01:01,29,"LINESTRING (-76.66535 39.30052, -76.66519 39.2...",TransportMode.BUS,7.88,1.02
18,245101607002002,245100402001036,4,2,TransportMode.WALK,2023-04-10 08:21:34,90.732,0 days 00:01:30,0 days 00:01:08,None,"LINESTRING (-76.67215 39.28185, -76.67214 39.2...",TransportMode.WALK,1.50,1.13
19,245101607002002,245100402001036,4,3,TransportMode.BUS,2023-04-10 08:27:50,NaN,0 days 00:12:20,0 days 00:05:38,CityLink PURPLE,"LINESTRING (-76.67263 39.28091, -76.66842 39.2...",TransportMode.BUS,12.33,5.63
20,245101607002002,245100402001036,4,4,TransportMode.WALK,2023-04-10 08:41:10,801.863,0 days 00:13:31,0 days 00:00:00,None,"LINESTRING (-76.62672 39.28670, -76.62668 39.2...",TransportMode.WALK,13.52,0.00


In [4]:
city_map

In [5]:
options, travel_details, city_map = compute_itineraries(transport_network_after_newline, '245101607002002', '245100402001036')
print(options.min())
print(travel_details)

34.9
            from_id            to_id  option  segment      transport_mode  \
56  245101607002002  245100402001036      16        0  TransportMode.WALK   
57  245101607002002  245100402001036      16        1  TransportMode.TRAM   
58  245101607002002  245100402001036      16        2  TransportMode.WALK   

        departure_time  distance     travel_time       wait_time     route  \
56 2023-04-10 08:05:59   842.715 0 days 00:14:14 0 days 00:00:00      None   
57 2023-04-10 08:23:36       NaN 0 days 00:08:19 0 days 00:01:07  Red Line   
58 2023-04-10 08:32:55   666.844 0 days 00:11:14 0 days 00:00:00      None   

                                             geometry                mode  \
56  LINESTRING (-76.66697 39.30002, -76.66587 39.3...  TransportMode.WALK   
57  LINESTRING (-76.66416 39.29445, -76.65333 39.2...  TransportMode.TRAM   
58  LINESTRING (-76.62672 39.28670, -76.62668 39.2...  TransportMode.WALK   

    travel time (min)  wait time (min)  
56              14.23  

In [6]:
city_map

# Baron
Baron travels from block 245102803011001 to 245100401002008

In [7]:
options, travel_details, city_map = compute_itineraries(transport_network_before_newline, '245102803011001', '245100401002008')
print(options.min())
print(travel_details)

63.97
             from_id            to_id  option  segment      transport_mode  \
179  245102803011001  245100401002008      33        0  TransportMode.WALK   
180  245102803011001  245100401002008      33        1   TransportMode.BUS   
181  245102803011001  245100401002008      33        2  TransportMode.WALK   
182  245102803011001  245100401002008      33        3   TransportMode.BUS   
183  245102803011001  245100401002008      33        4  TransportMode.WALK   
184  245102803011001  245100401002008      33        5   TransportMode.BUS   
185  245102803011001  245100401002008      33        6  TransportMode.WALK   

         departure_time  distance     travel_time       wait_time route  \
179 2023-04-10 08:00:06   260.946 0 days 00:04:22 0 days 00:00:00  None   
180 2023-04-10 08:06:16       NaN 0 days 00:02:15 0 days 00:01:02    34   
181 2023-04-10 08:09:31    83.286 0 days 00:01:21 0 days 00:01:14  None   
182 2023-04-10 08:13:44       NaN 0 days 00:37:00 0 days 00:03:38    

In [8]:
city_map

In [9]:
options, travel_details, city_map = compute_itineraries(transport_network_after_newline, '245102803011001', '245100401002008')
print(options.min())
print(travel_details)

43.28
            from_id            to_id  option  segment      transport_mode  \
61  245102803011001  245100401002008      21        0  TransportMode.WALK   
62  245102803011001  245100401002008      21        1   TransportMode.BUS   
63  245102803011001  245100401002008      21        2  TransportMode.WALK   
64  245102803011001  245100401002008      21        3  TransportMode.TRAM   
65  245102803011001  245100401002008      21        4  TransportMode.WALK   

        departure_time  distance     travel_time       wait_time     route  \
61 2023-04-10 08:00:06   260.946 0 days 00:04:22 0 days 00:00:00      None   
62 2023-04-10 08:06:16       NaN 0 days 00:01:24 0 days 00:01:02        34   
63 2023-04-10 08:08:40   335.549 0 days 00:05:35 0 days 00:02:46      None   
64 2023-04-10 08:20:35       NaN 0 days 00:19:20 0 days 00:05:34  Red Line   
65 2023-04-10 08:48:55   191.730 0 days 00:03:14 0 days 00:00:00      None   

                                             geometry         

In [10]:
city_map

# Cory 
Cory travels from block 245102608001020 to 245100401002008

In [11]:
options, travel_details, city_map = compute_itineraries(transport_network_before_newline, '245102608001020', '245100401002008', center=[39.299236, -76.609383])
print(options.min())
print(travel_details)

36.099999999999994
            from_id            to_id  option  segment      transport_mode  \
31  245102608001020  245100401002008      11        0  TransportMode.WALK   
32  245102608001020  245100401002008      11        1   TransportMode.BUS   
33  245102608001020  245100401002008      11        2  TransportMode.WALK   

        departure_time  distance     travel_time       wait_time  \
31 2023-04-10 08:09:31   299.451 0 days 00:04:57 0 days 00:00:00   
32 2023-04-10 08:08:02       NaN 0 days 00:20:45 0 days 00:01:03   
33 2023-04-10 08:29:47   554.500 0 days 00:09:21 0 days 00:00:00   

              route                                           geometry  \
31             None  LINESTRING (-76.56310 39.28982, -76.56314 39.2...   
32  CityLink ORANGE  LINESTRING (-76.56324 39.29163, -76.56548 39.2...   
33             None  LINESTRING (-76.61926 39.28690, -76.61816 39.2...   

                  mode  travel time (min)  wait time (min)  
31  TransportMode.WALK               4.95

In [12]:
city_map

In [13]:
options, travel_details, city_map = compute_itineraries(transport_network_after_newline, '245102608001020', '245100401002008', center=[39.299236, -76.609383])
print(options.min())
print(travel_details)

29.36
            from_id            to_id  option  segment      transport_mode  \
70  245102608001020  245100401002008      24        0  TransportMode.WALK   
71  245102608001020  245100401002008      24        1  TransportMode.TRAM   
72  245102608001020  245100401002008      24        2  TransportMode.WALK   

        departure_time  distance     travel_time       wait_time     route  \
70 2023-04-10 08:05:59   631.083 0 days 00:10:32 0 days 00:00:00      None   
71 2023-04-10 08:16:09       NaN 0 days 00:14:31 0 days 00:01:05  Red Line   
72 2023-04-10 08:39:40   191.730 0 days 00:03:14 0 days 00:00:00      None   

                                             geometry                mode  \
70  LINESTRING (-76.56310 39.28982, -76.56314 39.2...  TransportMode.WALK   
71  LINESTRING (-76.56141 39.28716, -76.56587 39.2...  TransportMode.TRAM   
72  LINESTRING (-76.61926 39.28690, -76.61816 39.2...  TransportMode.WALK   

    travel time (min)  wait time (min)  
70              10.53 

In [14]:
city_map

# Darrell
Darrell travels from block 245102007015006 to 245102605013028

In [15]:
options, travel_details, city_map = compute_itineraries(transport_network_before_newline, '245102007015006', '245102605013028', center=[39.299236, -76.609383])
print(options.min())
print(travel_details)

66.0
             from_id            to_id  option  segment      transport_mode  \
109  245102007015006  245102605013028      26        0  TransportMode.WALK   
110  245102007015006  245102605013028      26        1   TransportMode.BUS   
111  245102007015006  245102605013028      26        2   TransportMode.BUS   
112  245102007015006  245102605013028      26        3  TransportMode.WALK   

         departure_time  distance     travel_time       wait_time  \
109 2023-04-10 08:05:59   308.804 0 days 00:05:07 0 days 00:00:00   
110 2023-04-10 08:13:53       NaN 0 days 00:28:55 0 days 00:01:02   
111 2023-04-10 08:43:49       NaN 0 days 00:17:26 0 days 00:01:01   
112 2023-04-10 09:02:15   759.217 0 days 00:12:29 0 days 00:00:00   

               route                                           geometry  \
109             None  LINESTRING (-76.67707 39.29302, -76.67705 39.2...   
110    CityLink BLUE  LINESTRING (-76.67852 39.29381, -76.67587 39.2...   
111  CityLink ORANGE  LINESTRING 

In [16]:
city_map

In [17]:
options, travel_details, city_map = compute_itineraries(transport_network_after_newline, '245102007015006', '245102605013028', center=[39.299236, -76.609383])
print(options.min())
print(travel_details)

54.8
             from_id            to_id  option  segment      transport_mode  \
171  245102007015006  245102605013028      35        0  TransportMode.WALK   
172  245102007015006  245102605013028      35        1  TransportMode.TRAM   
173  245102007015006  245102605013028      35        2  TransportMode.WALK   
174  245102007015006  245102605013028      35        3   TransportMode.BUS   
175  245102007015006  245102605013028      35        4  TransportMode.WALK   

         departure_time  distance     travel_time       wait_time  \
171 2023-04-10 08:05:59   298.662 0 days 00:04:57 0 days 00:00:00   
172 2023-04-10 08:13:19       NaN 0 days 00:27:32 0 days 00:01:02   
173 2023-04-10 08:41:51   134.356 0 days 00:02:16 0 days 00:01:03   
174 2023-04-10 08:44:59       NaN 0 days 00:03:40 0 days 00:01:49   
175 2023-04-10 08:49:39   759.217 0 days 00:12:29 0 days 00:00:00   

               route                                           geometry  \
171             None  LINESTRING (-7

In [18]:
city_map